In [ ]:
%matplotlib inline 

import matplotlib.pyplot as plt
import numpy as np
from scipy.constants import c

from axiprop.utils import laser_from_fu, mirror_parabolic
from axiprop.lib import PropagatorSymmetric
from axiprop.lib import PropagatorResampling
from axiprop.lib import PropagatorResamplingFresnel

## Laser and focussing system parameters

In [ ]:
lambda0 = 0.8e-6              # wavelength [m]
tau_fwhm = 35e-15             # FWHM duration (intensity) [s]
R_las = 10e-3                 # Radial size of laser [m]

tau = tau_fwhm / (2*np.log(2))**0.5
k0 = 2 * np.pi / lambda0

def fu_laser(kz, r):
    """
    Gaussian spot with the Gaussian temporal profile
    """
    profile_r = np.exp( -(r/R_las)**2 ) * (r<2.5*R_las)
    profile_kz = np.exp( -( (kz-k0) * c * tau / 2 )**2 )
    return profile_r * profile_kz

f_N = 400                      # f-number of the parabolic mirror
f0 = 2 * R_las * f_N          # focal length

## Simulation and  numerical domain 

In [ ]:
L_kz = 8 / (c*tau)               # full spectral width [1/s] 
Nkz = 12                         # number of kz modes (1 for quick demonstration)
w0 = 2/np.pi * lambda0 * f_N     # beam waist
L_R = np.pi * w0**2/lambda0      # Rayleigh length
Rmax = 2.6 * R_las               # total radius [m]
Rmax_new = 16 * w0               # new radial grid size around focus

## Propagator

### alternatively try PropagatorSymmetric (needs much larger Nr)

### or PropagatorResamplingFresnel (paraxial)

In [ ]:
dr = w0/2                        # estimate grid resolution
Nr = int( Rmax / dr )            # total radial grid number
Nr_new = 256                     # new radial grid number around focus

prop = PropagatorResamplingFresnel((Rmax, Nr), (k0, L_kz,  Nkz), (Rmax_new, Nr_new))

## Single-step simulation

- CREATE THE LASER
- REFLECT IT FROM PARABOLIC MIRROR
- PROPAGATE TO THE FOCUS

In [ ]:
A0 = laser_from_fu( fu_laser, prop.kz, prop.r, normalize=True )
A0 /=  (R_las/w0)
A0 = A0 * mirror_parabolic( f0, prop.kz, prop.r )
A0 = prop.step(A0, f0)

In [ ]:
val = (np.abs(A0)**2).sum(0)
val = np.r_[val[::-1], val]
coord = np.r_[-prop.r_new[::-1], prop.r_new]/w0

plt.figure()
plt.plot(coord, val, c='k', lw=1.5)
plt.plot(coord, np.exp(-2*coord**2), '--', c='r')

plt.ylabel('Intensity (a.u.)', fontsize=12)
plt.xlabel('$R/w_0$', fontsize=12)

plt.xlim(-3,3)
plt.ylim(0,);

## Multi-step simulation

- CREATE THE LASER
- REFLECT IT FROM THE PARABOLIC MIRROR
- PROPAGATE AROUND THE FOCUS

In [ ]:
A0 = laser_from_fu( fu_laser, prop.kz, prop.r,normalize=True )
A0 /=  (R_las/w0)
A0 = A0 * mirror_parabolic( f0, prop.kz, prop.r )

Nsteps = 80
Distance = 8 * L_R

dz =  Distance / Nsteps
zsteps = Nsteps * [dz,]
zsteps[0] = f0 - Distance/2
z_axis = np.cumsum(zsteps)
R2 = (prop.r_new/w0)**2

A_multi = prop.steps(A0, z_axis=z_axis)

In [ ]:
IntesityPropagation = np.zeros((Nsteps, Nr_new))
waist = np.zeros((Nsteps))

for istep in range(Nsteps):
    val = (np.abs(A_multi[istep])**2).sum(0)
    IntesityPropagation[istep,:] = val
    waist[istep] = 2*np.average(R2, weights=val)**0.5

IntesityPropagation = np.array(IntesityPropagation)
FieldPropagation = IntesityPropagation**0.5

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(8,4), tight_layout=True, dpi=100)

ax.imshow(FieldPropagation.T, 
           extent=[ zsteps[0]*1e3, sum(zsteps)*1e3, 0, prop.r_new.max()/w0 ],
           aspect='auto', origin='lower', interpolation='gaussian',
           cmap=plt.cm.Greys_r)

ax.plot(z_axis*1e3, waist, c='w', lw=1.5)
ax.plot(z_axis*1e3, ( 1 + ((z_axis-f0)/L_R)**2 )**.5, ':', c='w', lw=1.5 )
ax.legend(('Beam waist',))


plt.xticks(np.array([f0-2*L_R, f0, f0+2*L_R])*1e3, 
           (r'$f_0-2L_R$', r'$f_0$', r'$f_0+2L_R$',), color='b')

plt.yticks(np.array([1,3]), color='b')

plt.title('Optical field amplitude (a.u.)',fontsize=12, color='b')
plt.xlabel('Distance', fontsize=12, color='b')
plt.ylabel('$R/w_0$', fontsize=12, color='b')
plt.ylim(0, 4.8);
#plt.savefig('./example_figure.png');